In [ ]:
%pip install mesa

In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET

from model import CampusModel
from visualization import draw

In [2]:
G1 = nx.read_graphml("data/distances-graph.graphml")
G2 = nx.read_graphml("data/width-graph.graphml")

G = G1.copy()

In [3]:
tree = ET.parse("data/distances-graph.graphml")
root = tree.getroot()

ns = {
    "g": "http://graphml.graphdrawing.org/xmlns",
    "y": "http://www.yworks.com/xml/graphml"
}

nodeTypes = {'#FFCC00': 'building',
             '#33CCCC': 'court',
             '#808080': 'temporary-point',
             '#FF0000': 'restaurant',
             '#333333': 'parking',
             '#993366': 'entry/exit',
             '#FF00FF': 'outpatient',
             '#008000': 'bank',
             '#FFCC99': 'bathroom',
             '#008080': 'turnstile'}

nodeAdditionalInformation = {}

for node in root.findall(".//g:node", ns):
    nodeID = node.attrib['id']
    for data in node:
        for ShapeNode in data:
            for x in ShapeNode:
                if 'Fill' in str(x):
                    nodeColor = x.attrib['color']
                    nodeAdditionalInformation[nodeID] = nodeTypes[nodeColor]

for id, data in G.nodes(data=True):
    data['type'] = nodeAdditionalInformation[id]

In [4]:
for u, v, data in G.edges(data=True):
    data['distance'] = int(data['label'])
    data["width"] = int(G2.edges[u, v]['label'])

In [ ]:
G = G.to_undirected()
restaurants = [id for id, data in G.nodes(data=True) if data["type"] in ('restaurant')]
bathrooms = [id for id, data in G.nodes(data=True) if data["type"] in ('bathroom')]

6


In [ ]:
model = CampusModel(G, 1, restaurants, bathrooms)

In [ ]:
pos = {
    n: (float(data['x']), float(data['y'])) for n, data in G.nodes(data=True)
}

plt.ion()

time = 20000

draw(model, pos)
for i in range(time):
    model.step()
    result = model.anyone_moved
    if result:
        print(f"\nTempo: {i}\n")
        draw(model, pos)

plt.ioff()
plt.show()